<a href="https://colab.research.google.com/github/hakim-cyber/Comp-Science-for-physics-and-chemistry/blob/main/PW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Data1.txt to Data1.txt
Saving Data2.txt to Data2.txt
Saving Data4.dat to Data4.dat
Saving Data5.txt to Data5.txt
Saving Data3.txt to Data3.txt
Saving Data6.txt to Data6.txt


In [7]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


PAH = [
    ("Naphthalene",         200.9),
    ("Anthracene",            5.8),
    ("Benzopyrene",          38.6),
    ("Pyrene",              516.2),
    ("Chrysene",             57.8),
    ("Benzofluoranthene",     8.9),
]
TAUS = np.array([tau for _, tau in PAH], dtype=float)

DAT_SCALE = 10.0 / 65535.0

def multi_exp(t, *A):
    A = np.asarray(A)
    return np.sum(A * np.exp(-t[:, None] / TAUS[None, :]), axis=1)


def list_datasets(root="/content"):
    p = Path(root)
    return sorted([q for q in p.iterdir() if q.suffix.lower() in (".txt", ".dat")], key=lambda x: x.name.lower())

def load_txt(path: Path):
    y = np.loadtxt(path, dtype=float, ndmin=1)
    return y.astype(float)

def load_dat(path: Path):

    y_raw = np.fromfile(path, dtype=np.uint16)
    return y_raw.astype(float) * DAT_SCALE

def build_time(n, t0=0.0, t1=1000.0):
    return np.linspace(t0, t1, n)


def fit_amplitudes(t, y):
    ncomp = len(TAUS)
    A0 = np.full(ncomp, max(1e-6, float(np.max(y)) / ncomp))
    bounds = (np.zeros(ncomp), np.full(ncomp, np.inf))
    popt, pcov = curve_fit(
        multi_exp, t, y, p0=A0, bounds=bounds, maxfev=200000
    )
    perr = np.sqrt(np.diag(pcov)) if (pcov is not None and pcov.size) else np.full_like(popt, np.nan)
    return popt, perr

# ---- Run over all files in /content ----
def process_all(root="/content", png_prefix="fit_", summary_csv="composition_summary.csv"):
    files = list_datasets(root)
    if not files:
        print("No .txt or .dat files found in /content.")
        return None

    rows = []
    for f in files:
        if f.suffix.lower() == ".txt":
            y = load_txt(f); source = "txt"
        else:
            y = load_dat(f); source = "dat"

        t = build_time(len(y))
        A, Aerr = fit_amplitudes(t, y)
        y_fit = multi_exp(t, *A)

        # Save plot
        plt.figure(figsize=(8,5))
        plt.plot(t, y, lw=1.0, label="Measured")
        plt.plot(t, y_fit, lw=1.6, label="Fit")
        plt.xlabel("Time (ns)"); plt.ylabel("Fluorescence (arb.)")
        plt.title(f"{f.stem} — nonnegative fit")
        plt.legend()
        out_png = Path(root) / f"{png_prefix}{f.stem}.png"
        plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()
        print(f"Saved {out_png}")

        # Row for CSV
        row = {"file": f.name, "source": source}
        for (label, tau), a, e in zip(PAH, A, Aerr):
            row[f"A_{label}"] = float(a)
            row[f"±_{label}"] = float(e) if np.isfinite(e) else np.nan
        # simple presence flag
        present = [PAH[i][0] for i, a in enumerate(A) if a > 1e-3]
        row["present(>1e-3)"] = ", ".join(present) if present else "(none)"
        rows.append(row)

        # Console summary
        print(f"\n=== {f.name} ===")
        for (label, tau), a, e in zip(PAH, A, Aerr):
            e_str = "nan" if not np.isfinite(e) else f"{e:.3g}"
            print(f"  {label:22s} τ={tau:6.1f} ns : A={a:.5g} (±{e_str})")
        print("  Present >", 1e-3, ":", row["present(>1e-3)"])

    # Save/preview CSV
    df = pd.DataFrame(rows)
    out_csv = Path(root) / summary_csv
    df.to_csv(out_csv, index=False)
    print("\nSaved summary:", out_csv)
    try:
        from IPython.display import display
        display(df)
    except:
        pass
    return df

# Execute
_ = process_all("/content")

Saved /content/fit_Data1.png

=== Data1.txt ===
  Naphthalene            τ= 200.9 ns : A=0.99999 (±1.72e-07)
  Anthracene             τ=   5.8 ns : A=3.9403e-06 (±1.11e-06)
  Benzopyrene            τ=  38.6 ns : A=6.7845e-06 (±9.73e-07)
  Pyrene                 τ= 516.2 ns : A=4.1614e-06 (±5.48e-08)
  Chrysene               τ=  57.8 ns : A=9.8879e-07 (±8.2e-07)
  Benzofluoranthene      τ=   8.9 ns : A=3.0221e-06 (±1.24e-06)
  Present > 0.001 : Naphthalene
Saved /content/fit_Data2.png

=== Data2.txt ===
  Naphthalene            τ= 200.9 ns : A=1.9161e-06 (±1.03e-07)
  Anthracene             τ=   5.8 ns : A=2 (±6.64e-07)
  Benzopyrene            τ=  38.6 ns : A=3.4869e-06 (±5.8e-07)
  Pyrene                 τ= 516.2 ns : A=3 (±3.27e-08)
  Chrysene               τ=  57.8 ns : A=2.6726e-07 (±4.89e-07)
  Benzofluoranthene      τ=   8.9 ns : A=5.4508e-06 (±7.4e-07)
  Present > 0.001 : Anthracene, Pyrene
Saved /content/fit_Data3.png

=== Data3.txt ===
  Naphthalene            τ= 200.9 ns : A=

,file,source,A_Naphthalene,±_Naphthalene,A_Anthracene,±_Anthracene,A_Benzopyrene,±_Benzopyrene,A_Pyrene,±_Pyrene,A_Chrysene,±_Chrysene,A_Benzofluoranthene,±_Benzofluoranthene,present(>1e-3)
0,Data1.txt,txt,9.999909e-01,1.718636e-07,3.940270e-06,1.112580e-06,6.784452e-06,9.731675e-07,4.161374e-06,5.481585e-08,9.887863e-07,8.196431e-07,3.022070e-06,1.240589e-06,Naphthalene
1,Data2.txt,txt,1.916075e-06,1.025131e-07,1.999986e+00,6.636309e-07,3.486939e-06,5.804744e-07,2.999998e+00,3.269653e-08,2.672552e-07,4.889003e-07,5.450769e-06,7.399856e-07,"Anthracene, Pyrene"
2,Data3.txt,txt,1.000000e+00,1.104572e-17,2.000000e+00,7.150580e-17,3.000000e+00,6.254574e-17,4.000000e+00,3.523030e-18,2.731888e-17,5.267868e-17,6.133907e-16,7.973296e-17,"Naphthalene, Anthracene, Benzopyrene, Pyrene"
3,Data4.dat,dat,4.999727e+00,1.199328e-05,1.085559e-07,7.763996e-05,1.439200e-10,6.791126e-05,3.243526e-13,3.825255e-06,3.000275e+00,5.719776e-05,4.382166e-08,8.657290e-05,"Naphthalene, Chrysene"
4,Data5.txt,txt,8.338908e-16,9.791144e-03,2.078096e-19,6.338415e-02,1.527896e+00,5.544178e-02,7.094843e-15,3.122884e-03,6.402776e-19,4.669543e-02,1.418323e+00,7.067688e-02,"Benzopyrene, Benzofluoranthene"
5,Data6.txt,txt,4.556491e-14,1.454475e-02,4.429836e-18,9.415723e-02,1.550811e+00,8.235883e-02,3.423480e-13,4.639047e-03,6.951837e-17,6.936611e-02,1.383265e+00,1.049906e-01,"Benzopyrene, Benzofluoranthene"


from matplotlib import pyplot as plt
_df_67['A_Naphthalene'].plot(kind='hist', bins=20, title='A_Naphthalene')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_68['±_Naphthalene'].plot(kind='hist', bins=20, title='±_Naphthalene')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_69['A_Anthracene'].plot(kind='hist', bins=20, title='A_Anthracene')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_85['source'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_85, x='A_Naphthalene', y='source', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_86['present(>1e-3)'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_86, x='A_Naphthalene', y='present(>1e-3)', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_87['file'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_87, x='±_Naphthalene', y='file', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)